In [1]:
import pandas as pd
import numpy as np
import rapidfuzz

In [2]:
state='NH'
occs_and_debt = pd.read_excel(r'C:\Users\david\OneDrive\Documents\GitHub\SPEOC-pt-1\Data\Post1790\NH\T652_R6_New_Hampshire_CD.xlsx',
                             header=10, usecols='L, M, N, O, P, Q, R, S') #occupations plus continental debt for conneticut

In [3]:
occs_and_debt = occs_and_debt.dropna(subset=['Occupation'])
print(occs_and_debt)

    Unnamed: 11 Occupation  Dollars  Cents  Dollars.1  Cents.1  Dollars.2  \
0            NH  Physician   1490.0   73.0      745.0     37.0      436.0   
1            NH      Widow   2100.0   48.0     1050.0     23.0      928.0   
2            NH  Treasurer    149.0   62.0       74.0     81.0        NaN   
3            NH    Esquire   4082.0    6.0     2041.0      3.0     2538.0   
4            NH   Merchant    968.0   23.0      484.0     12.0      611.0   
..          ...        ...      ...    ...        ...      ...        ...   
214          NH  Physician    187.0    2.0       93.0     51.0      146.0   
215          NH    Esquire     12.0   73.0        6.0     37.0        0.0   
216          NH     Trader      9.0   16.0        4.0     58.0       10.0   
217          MA     Trader     29.0    2.0       14.0     51.0       29.0   
218          MA   Merchant    293.0   93.0      146.0     96.0      109.0   

     Cents.2  
0       77.0  
1       46.0  
2        NaN  
3       45.0  


In [4]:
#exclude non states 
occs_and_debt = occs_and_debt.rename(columns={'Unnamed: 11':'State'})
occs_and_debt = occs_and_debt[occs_and_debt['State'].str.contains(state) == True].drop('State', axis=1)
print(occs_and_debt)

    Occupation  Dollars  Cents  Dollars.1  Cents.1  Dollars.2  Cents.2
0    Physician   1490.0   73.0      745.0     37.0      436.0     77.0
1        Widow   2100.0   48.0     1050.0     23.0      928.0     46.0
2    Treasurer    149.0   62.0       74.0     81.0        NaN      NaN
3      Esquire   4082.0    6.0     2041.0      3.0     2538.0     45.0
4     Merchant    968.0   23.0      484.0     12.0      611.0      2.0
..         ...      ...    ...        ...      ...        ...      ...
212   Merchant    106.0   83.0       53.0     41.0       96.0     27.0
213   Merchant     81.0   69.0       40.0     84.0      741.0     54.0
214  Physician    187.0    2.0       93.0     51.0      146.0     70.0
215    Esquire     12.0   73.0        6.0     37.0        0.0      0.0
216     Trader      9.0   16.0        4.0     58.0       10.0     16.0

[163 rows x 7 columns]


In [5]:
occurences_dict = occs_and_debt['Occupation'].value_counts().to_dict()
print(occurences_dict)

{'Esquire': 66, 'Merchant': 25, 'Physician': 18, 'Yeoman': 12, 'Widow': 9, 'Trader': 8, 'Clerk': 7, 'Gentleman': 6, 'Cordwainer': 2, 'Joiner': 2, 'Marniner': 2, 'Treasurer': 1, 'Goldsmith': 1, 'Spinster': 1, 'Baker': 1, 'Blacksmith': 1, 'Painter': 1}


In [6]:
totals = occs_and_debt.groupby('Occupation')[['Dollars', 'Dollars.1', 'Dollars.2', 'Cents', 'Cents.1', 'Cents.2']].apply(sum)
totals = totals.reset_index()
print(totals.to_string())

    Occupation  Dollars  Dollars.1  Dollars.2   Cents  Cents.1  Cents.2
0        Baker   2159.0     1079.0      588.0     4.0     52.0     18.0
1   Blacksmith     54.0       27.0       14.0    88.0     44.0     81.0
2        Clerk   2713.0     1355.0      975.0   260.0    280.0    308.0
3   Cordwainer    783.0      391.0      269.0     5.0     52.0    113.0
4      Esquire  67440.0    33746.0    36323.0  3289.0   3046.0   2684.0
5    Gentleman   3601.0     1799.0     1256.0   206.0    253.0    248.0
6    Goldsmith    871.0      435.0      252.0    32.0     66.0      0.0
7       Joiner    969.0      484.0      301.0    61.0     81.0    106.0
8     Marniner    784.0      391.0      576.0   162.0    192.0    142.0
9     Merchant  30061.0    15024.0    21153.0  1374.0   1338.0   1120.0
10     Painter     48.0       24.0       37.0    61.0     31.0     44.0
11   Physician   7102.0     3547.0     3153.0   943.0    825.0    757.0
12    Spinster     89.0       44.0       24.0    13.0     57.0  

In [7]:
occs_debt_wo_dups = totals.copy()
#use fuzzy string matching to remove any similar names like merchant and merchents. does not take into account longer titles.
occ_list = occs_debt_wo_dups['Occupation'].tolist()

for occ in occ_list:
    for occ_2 in occ_list:
        if occ != occ_2:
            if rapidfuzz.fuzz.ratio(occ, occ_2) >= 80 and occ != occ_2:
                #transfer amount in row that will be deleted to other row
                row_w_occ_2 = occs_debt_wo_dups.loc[occs_debt_wo_dups['Occupation'] == occ_2] #locate row that contains occupation soon to be deleted
                dollars_in_occ_2 = row_w_occ_2[['Dollar', 'Dollar.1', 'Dollar.2']].sum(axis=1).values #get total dollars
                cents_in_occ_2 = row_w_occ_2[['Cents', 'Cents.1', 'Cents.2']].sum(axis=1).values #get total cents
                dollars_in_occ_2 = dollars_in_occ_2[0] #stored in array. need to retrieve value
                cents_in_occ_2 = cents_in_occ_2[0] #stored in array. need to retrieve value  
                occs_debt_wo_dups.loc[(occs_debt_wo_dups.Occupation == occ), 'Dollar'] += dollars_in_occ_2 #add dollars to occupation 1
                occs_debt_wo_dups.loc[(occs_debt_wo_dups.Occupation == occ), 'Cents'] += cents_in_occ_2 #add cents to occupation 1
                 
                #remove value from occurences dictionary and add one to other occupation
                occurences_dict[occs_debt_wo_dups.loc[(occs_debt_wo_dups.Occupation == occ), 'Occupation'].values[0]] += occurences_dict[occs_debt_wo_dups.loc[(occs_debt_wo_dups.Occupation == occ_2), 'Occupation'].values[0]] 
                occurences_dict.pop(occ_2)
                
                #remove row
                occs_debt_wo_dups.drop(occs_debt_wo_dups[occs_debt_wo_dups['Occupation'] == occ_2].index, inplace=True) 
                occ_list.remove(occ_2)

occs_debt_wo_dups = occs_debt_wo_dups.reset_index(drop=True) #reset indexes 0-65 

#take the first two words of every job if applicable and remove any duplicate titles. handles longer titles 
two_word_occs = {}
#retrieve all two worded occupations
for occ in occ_list:
    if len(occ.split()) >= 2: #only account for words that are actually more than two words long
        occ_two_word = occ.split()[0] + " " + occ.split()[1] #get first two words
        two_word_occs[occ_list.index(occ)] = occ_two_word #append both words to dict

#retrieve any duplicates now 
flipped = {} 
for key, value in two_word_occs.items():
    flipped.setdefault(value, list()).append(key) #flip keys with values

keys_w_dups = [key for key, values in flipped.items() if len(values) > 1] #retrieve keys that have multiple values. store them in a list

print(occurences_dict)

{'Esquire': 66, 'Merchant': 25, 'Physician': 18, 'Yeoman': 12, 'Widow': 9, 'Trader': 8, 'Clerk': 7, 'Gentleman': 6, 'Cordwainer': 2, 'Joiner': 2, 'Marniner': 2, 'Treasurer': 1, 'Goldsmith': 1, 'Spinster': 1, 'Baker': 1, 'Blacksmith': 1, 'Painter': 1}


In [8]:
print(occs_debt_wo_dups.to_string())

    Occupation  Dollars  Dollars.1  Dollars.2   Cents  Cents.1  Cents.2
0        Baker   2159.0     1079.0      588.0     4.0     52.0     18.0
1   Blacksmith     54.0       27.0       14.0    88.0     44.0     81.0
2        Clerk   2713.0     1355.0      975.0   260.0    280.0    308.0
3   Cordwainer    783.0      391.0      269.0     5.0     52.0    113.0
4      Esquire  67440.0    33746.0    36323.0  3289.0   3046.0   2684.0
5    Gentleman   3601.0     1799.0     1256.0   206.0    253.0    248.0
6    Goldsmith    871.0      435.0      252.0    32.0     66.0      0.0
7       Joiner    969.0      484.0      301.0    61.0     81.0    106.0
8     Marniner    784.0      391.0      576.0   162.0    192.0    142.0
9     Merchant  30061.0    15024.0    21153.0  1374.0   1338.0   1120.0
10     Painter     48.0       24.0       37.0    61.0     31.0     44.0
11   Physician   7102.0     3547.0     3153.0   943.0    825.0    757.0
12    Spinster     89.0       44.0       24.0    13.0     57.0  

In [9]:
print(flipped)
print("\n\n")
print(keys_w_dups)

{}



[]


In [10]:
print(occurences_dict)

for occ in keys_w_dups:
    i = 0
    while i < len(flipped[occ]):
        index = flipped[occ][i]
        if len(flipped[occ]) > 1: #make sure to keep one index in the list. don't want to delete ALL occurences
            duplicate_row = occs_debt_wo_dups.loc[[index]] #return row that has index
            dollars_in_dup_row = duplicate_row[['Dollar', 'Dollar.1', 'Dollar.2']].sum(axis=1).values #sum all dollars in row                
            cents_in_dup_row = duplicate_row[['Cents', 'Cents.1', 'Cents.2']].sum(axis=1).values #return total cents in row
            dollars_dup_row = dollars_in_dup_row[0] #stored in array. need to retrieve value
            cents_dup_row = cents_in_dup_row[0] #stored in array. need to retrieve value

            last_index = flipped[occ][len(flipped[occ]) - 1] #get last value in list. we will add dollars and cents here
                        
            occs_debt_wo_dups.loc[last_index, 'Dollar'] += dollars_dup_row #add dollars from one occupation to the other
            occs_debt_wo_dups.loc[last_index, 'Cents'] += cents_dup_row #add cents from one occupation to the other
            
            #remove value from occurences dictionary and add one to other occupation
            
            #print("Removed: " + str(occs_debt_wo_dups.loc[index, 'Occupation']))
            #print('Kept: ' + str(occs_debt_wo_dups.loc[last_index, 'Occupation']))
            
            print(occs_debt_wo_dups.loc[last_index, 'Occupation'])
            occurences_dict[occs_debt_wo_dups.loc[last_index, 'Occupation']] += occurences_dict[occs_debt_wo_dups.loc[index, 'Occupation']] 
            occurences_dict.pop(occs_debt_wo_dups.loc[index, 'Occupation'])
            
            print("Length: " + str(len(flipped[occ])))
            print("Last Index: " + str(last_index) + " Index: " + str(index))
            print(flipped[occ])
            print("\n\n")
                        
            occs_debt_wo_dups.drop(index, inplace=True) #remove duplicate occupation now
            flipped[occ].remove(index) #remove from list
            
            #print(occurences_dict)
            #print("\n\n")
        else:
            i += 1

occs_debt_wo_dups = occs_debt_wo_dups.reset_index(drop=True) #reset indexes 0-...

{'Esquire': 66, 'Merchant': 25, 'Physician': 18, 'Yeoman': 12, 'Widow': 9, 'Trader': 8, 'Clerk': 7, 'Gentleman': 6, 'Cordwainer': 2, 'Joiner': 2, 'Marniner': 2, 'Treasurer': 1, 'Goldsmith': 1, 'Spinster': 1, 'Baker': 1, 'Blacksmith': 1, 'Painter': 1}


In [11]:
print(occs_debt_wo_dups.to_string())

    Occupation  Dollars  Dollars.1  Dollars.2   Cents  Cents.1  Cents.2
0        Baker   2159.0     1079.0      588.0     4.0     52.0     18.0
1   Blacksmith     54.0       27.0       14.0    88.0     44.0     81.0
2        Clerk   2713.0     1355.0      975.0   260.0    280.0    308.0
3   Cordwainer    783.0      391.0      269.0     5.0     52.0    113.0
4      Esquire  67440.0    33746.0    36323.0  3289.0   3046.0   2684.0
5    Gentleman   3601.0     1799.0     1256.0   206.0    253.0    248.0
6    Goldsmith    871.0      435.0      252.0    32.0     66.0      0.0
7       Joiner    969.0      484.0      301.0    61.0     81.0    106.0
8     Marniner    784.0      391.0      576.0   162.0    192.0    142.0
9     Merchant  30061.0    15024.0    21153.0  1374.0   1338.0   1120.0
10     Painter     48.0       24.0       37.0    61.0     31.0     44.0
11   Physician   7102.0     3547.0     3153.0   943.0    825.0    757.0
12    Spinster     89.0       44.0       24.0    13.0     57.0  

In [12]:
print(flipped)

{}


In [13]:
print(occurences_dict)

{'Esquire': 66, 'Merchant': 25, 'Physician': 18, 'Yeoman': 12, 'Widow': 9, 'Trader': 8, 'Clerk': 7, 'Gentleman': 6, 'Cordwainer': 2, 'Joiner': 2, 'Marniner': 2, 'Treasurer': 1, 'Goldsmith': 1, 'Spinster': 1, 'Baker': 1, 'Blacksmith': 1, 'Painter': 1}


In [14]:
exceptions = [] #applicable only to a few states

In [15]:
#get the first word from every occupation and store any duplicates in a dictionary
occ_list = occs_debt_wo_dups['Occupation'].to_list()
occupations = []
one_word_occs = {}
flipped_one_w = {}

for occ in occ_list:
    if occ not in exceptions:
        first_word = occ.split()[0]
        one_word_occs[occ_list.index(occ)] = first_word 
        
#flip dictionary
for key, value in one_word_occs.items():
    flipped_one_w.setdefault(value, list()).append(key) #flip keys with values

In [16]:
#use fuzzy string matching 
flipped_list = list(flipped_one_w) #convert to list. needed in order to iterate through
for occ in flipped_list:
    for occ_2 in flipped_list:
        if occ_2 != occ and rapidfuzz.fuzz.ratio(occ, occ_2) >= 80:
            flipped_one_w[occ].extend(flipped_one_w[occ_2]) #combine lists
            flipped_one_w.pop(occ_2) #remove second occurence of the same occupation 
            flipped_list.remove(occ_2) #remove from list 
            
keys_w_dups = [key for key, values in flipped_one_w.items() if len(values) > 1] #retrieve keys that have multiple values. store them in a list
         
print(flipped_one_w)
print("\n\n")
print(keys_w_dups)

{'Baker': [0], 'Blacksmith': [1], 'Clerk': [2], 'Cordwainer': [3], 'Esquire': [4], 'Gentleman': [5], 'Goldsmith': [6], 'Joiner': [7], 'Marniner': [8], 'Merchant': [9], 'Painter': [10], 'Physician': [11], 'Spinster': [12], 'Trader': [13], 'Treasurer': [14], 'Widow': [15], 'Yeoman': [16]}



[]


In [17]:
#remove duplicates
for occ in keys_w_dups:
    i = 0
    while i < len(flipped_one_w[occ]):
        if len(flipped_one_w[occ]) > 1:
            
            index = flipped_one_w[occ][i]
            last_index = flipped_one_w[occ][len(flipped_one_w[occ]) - 1] #get last value in list. we will add dollars and cents here
            duplicate_row = occs_debt_wo_dups.loc[[index]] #returns row that has index
            dollars_in_dup_row = duplicate_row[['Dollar', 'Dollar.1', 'Dollar.2']].sum(axis=1).values[0]
            cents_in_dup_row = duplicate_row[['Cents', 'Cents.1', 'Cents.2']].sum(axis=1).values[0]
            
            occs_debt_wo_dups.loc[last_index, 'Dollar'] += dollars_in_dup_row
            occs_debt_wo_dups.loc[last_index, 'Cents'] += cents_dup_row
            
            occurences_dict[occs_debt_wo_dups.loc[last_index, 'Occupation']] += occurences_dict[occs_debt_wo_dups.loc[index, 'Occupation']] 
            occurences_dict.pop(occs_debt_wo_dups.loc[index, 'Occupation'])

            occs_debt_wo_dups.drop(index, inplace=True)
            flipped_one_w[occ].remove(index)
            
        else:
            i += 1

occs_debt_wo_dups = occs_debt_wo_dups.reset_index(drop=True) #reset indexes 0-...

print(occs_debt_wo_dups.to_string())

    Occupation  Dollars  Dollars.1  Dollars.2   Cents  Cents.1  Cents.2
0        Baker   2159.0     1079.0      588.0     4.0     52.0     18.0
1   Blacksmith     54.0       27.0       14.0    88.0     44.0     81.0
2        Clerk   2713.0     1355.0      975.0   260.0    280.0    308.0
3   Cordwainer    783.0      391.0      269.0     5.0     52.0    113.0
4      Esquire  67440.0    33746.0    36323.0  3289.0   3046.0   2684.0
5    Gentleman   3601.0     1799.0     1256.0   206.0    253.0    248.0
6    Goldsmith    871.0      435.0      252.0    32.0     66.0      0.0
7       Joiner    969.0      484.0      301.0    61.0     81.0    106.0
8     Marniner    784.0      391.0      576.0   162.0    192.0    142.0
9     Merchant  30061.0    15024.0    21153.0  1374.0   1338.0   1120.0
10     Painter     48.0       24.0       37.0    61.0     31.0     44.0
11   Physician   7102.0     3547.0     3153.0   943.0    825.0    757.0
12    Spinster     89.0       44.0       24.0    13.0     57.0  

In [18]:
occs_debt_wo_dups[['Cents', 'Cents.1', 'Cents.2']] = occs_debt_wo_dups[['Cents', 'Cents.1', 'Cents.2']].div(100)
print(occs_debt_wo_dups.to_string())

    Occupation  Dollars  Dollars.1  Dollars.2  Cents  Cents.1  Cents.2
0        Baker   2159.0     1079.0      588.0   0.04     0.52     0.18
1   Blacksmith     54.0       27.0       14.0   0.88     0.44     0.81
2        Clerk   2713.0     1355.0      975.0   2.60     2.80     3.08
3   Cordwainer    783.0      391.0      269.0   0.05     0.52     1.13
4      Esquire  67440.0    33746.0    36323.0  32.89    30.46    26.84
5    Gentleman   3601.0     1799.0     1256.0   2.06     2.53     2.48
6    Goldsmith    871.0      435.0      252.0   0.32     0.66     0.00
7       Joiner    969.0      484.0      301.0   0.61     0.81     1.06
8     Marniner    784.0      391.0      576.0   1.62     1.92     1.42
9     Merchant  30061.0    15024.0    21153.0  13.74    13.38    11.20
10     Painter     48.0       24.0       37.0   0.61     0.31     0.44
11   Physician   7102.0     3547.0     3153.0   9.43     8.25     7.57
12    Spinster     89.0       44.0       24.0   0.13     0.57     0.07
13    

In [19]:
occs_debt_wo_dups['Total'] = occs_debt_wo_dups[['Dollars', 'Dollars.1', 'Dollars.2', 'Cents', 'Cents.1', 'Cents.2']].sum(axis=1) #add totals to new column
print(occs_debt_wo_dups.to_string())

    Occupation  Dollars  Dollars.1  Dollars.2  Cents  Cents.1  Cents.2      Total
0        Baker   2159.0     1079.0      588.0   0.04     0.52     0.18    3826.74
1   Blacksmith     54.0       27.0       14.0   0.88     0.44     0.81      97.13
2        Clerk   2713.0     1355.0      975.0   2.60     2.80     3.08    5051.48
3   Cordwainer    783.0      391.0      269.0   0.05     0.52     1.13    1444.70
4      Esquire  67440.0    33746.0    36323.0  32.89    30.46    26.84  137599.19
5    Gentleman   3601.0     1799.0     1256.0   2.06     2.53     2.48    6663.07
6    Goldsmith    871.0      435.0      252.0   0.32     0.66     0.00    1558.98
7       Joiner    969.0      484.0      301.0   0.61     0.81     1.06    1756.48
8     Marniner    784.0      391.0      576.0   1.62     1.92     1.42    1755.96
9     Merchant  30061.0    15024.0    21153.0  13.74    13.38    11.20   66276.32
10     Painter     48.0       24.0       37.0   0.61     0.31     0.44     110.36
11   Physician  

In [20]:
occs_debt_wo_dups['# of People'] = occs_debt_wo_dups['Occupation'].map(occurences_dict) #add occurences to table
print(occs_debt_wo_dups[['Occupation', '# of People']])

    Occupation  # of People
0        Baker            1
1   Blacksmith            1
2        Clerk            7
3   Cordwainer            2
4      Esquire           66
5    Gentleman            6
6    Goldsmith            1
7       Joiner            2
8     Marniner            2
9     Merchant           25
10     Painter            1
11   Physician           18
12    Spinster            1
13      Trader            8
14   Treasurer            1
15       Widow            9
16      Yeoman           12


In [21]:
occs_debt_wo_dups['Average'] = occs_debt_wo_dups['Total'] / occs_debt_wo_dups['# of People'] #calculate average and add to new column
print(occs_debt_wo_dups[['Occupation', 'Total', '# of People', 'Average']].to_string())

    Occupation      Total  # of People      Average
0        Baker    3826.74            1  3826.740000
1   Blacksmith      97.13            1    97.130000
2        Clerk    5051.48            7   721.640000
3   Cordwainer    1444.70            2   722.350000
4      Esquire  137599.19           66  2084.836212
5    Gentleman    6663.07            6  1110.511667
6    Goldsmith    1558.98            1  1558.980000
7       Joiner    1756.48            2   878.240000
8     Marniner    1755.96            2   877.980000
9     Merchant   66276.32           25  2651.052800
10     Painter     110.36            1   110.360000
11   Physician   13827.25           18   768.180556
12    Spinster     157.77            1   157.770000
13      Trader    5953.28            8   744.160000
14   Treasurer     224.43            1   224.430000
15       Widow   16213.01            9  1801.445556
16      Yeoman    6565.79           12   547.149167


In [22]:
#convert to one word
exceptions = ['Society Committee', 'Society Treasurer', 'School Master', 'Inn Keeper', 'Leather Dresser', 'Post Rider', 'School Committee North Parish']
def first_word_only(occ):
    if 'School Committee' in occ:
        return 'School Committee'
    if occ not in exceptions:
        return occ.split()[0]
    else:
        return occ
    
occs_debt_wo_dups['Occupation'] = occs_debt_wo_dups['Occupation'].apply(first_word_only)
print(occs_debt_wo_dups.to_string())

    Occupation  Dollars  Dollars.1  Dollars.2  Cents  Cents.1  Cents.2      Total  # of People      Average
0        Baker   2159.0     1079.0      588.0   0.04     0.52     0.18    3826.74            1  3826.740000
1   Blacksmith     54.0       27.0       14.0   0.88     0.44     0.81      97.13            1    97.130000
2        Clerk   2713.0     1355.0      975.0   2.60     2.80     3.08    5051.48            7   721.640000
3   Cordwainer    783.0      391.0      269.0   0.05     0.52     1.13    1444.70            2   722.350000
4      Esquire  67440.0    33746.0    36323.0  32.89    30.46    26.84  137599.19           66  2084.836212
5    Gentleman   3601.0     1799.0     1256.0   2.06     2.53     2.48    6663.07            6  1110.511667
6    Goldsmith    871.0      435.0      252.0   0.32     0.66     0.00    1558.98            1  1558.980000
7       Joiner    969.0      484.0      301.0   0.61     0.81     1.06    1756.48            2   878.240000
8     Marniner    784.0     

In [23]:
total = occs_debt_wo_dups['Total'].sum()
occs_debt_wo_dups['% of Total Debt'] = round((occs_debt_wo_dups['Total'] / total) * 100, 3)
print(occs_debt_wo_dups.to_string())

    Occupation  Dollars  Dollars.1  Dollars.2  Cents  Cents.1  Cents.2      Total  # of People      Average  % of Total Debt
0        Baker   2159.0     1079.0      588.0   0.04     0.52     0.18    3826.74            1  3826.740000            1.422
1   Blacksmith     54.0       27.0       14.0   0.88     0.44     0.81      97.13            1    97.130000            0.036
2        Clerk   2713.0     1355.0      975.0   2.60     2.80     3.08    5051.48            7   721.640000            1.877
3   Cordwainer    783.0      391.0      269.0   0.05     0.52     1.13    1444.70            2   722.350000            0.537
4      Esquire  67440.0    33746.0    36323.0  32.89    30.46    26.84  137599.19           66  2084.836212           51.137
5    Gentleman   3601.0     1799.0     1256.0   2.06     2.53     2.48    6663.07            6  1110.511667            2.476
6    Goldsmith    871.0      435.0      252.0   0.32     0.66     0.00    1558.98            1  1558.980000            0.579


In [24]:
#final total debt held by people w/ no job listed
occs_and_debt = pd.read_excel(r"C:\Users\david\OneDrive\Documents\GitHub\SPEOC-pt-1\Data\Post1790\CT\CT_post1790_CD_ledger.xlsx",
                             header=13, usecols='L, N, O, AD, AE, AT, AU') #occupations plus continental debt for conneticut
no_occ = occs_and_debt[occs_and_debt['Occupation'].isna()]
num_of_people = len(no_occ)
no_occ = no_occ.reset_index(drop=True)
no_occ[['Cents', 'Cents.1', 'Cents.2']] = no_occ[['Cents', 'Cents.1', 'Cents.2']].div(100)
total_series = no_occ[['Dollar', 'Dollar.1', 'Dollar.2', 'Cents', 'Cents.1', 'Cents.2']].sum(axis=1)
total = total_series.sum()

occs_debt_wo_dups = occs_debt_wo_dups.append({'Occupation': 'None Listed', 'Total': total, '# of People': num_of_people, 'Average': total / num_of_people}, ignore_index=True)
print(occs_debt_wo_dups.to_string())

     Occupation  Dollars  Dollars.1  Dollars.2  Cents  Cents.1  Cents.2       Total  # of People      Average  % of Total Debt
0         Baker   2159.0     1079.0      588.0   0.04     0.52     0.18     3826.74            1  3826.740000            1.422
1    Blacksmith     54.0       27.0       14.0   0.88     0.44     0.81       97.13            1    97.130000            0.036
2         Clerk   2713.0     1355.0      975.0   2.60     2.80     3.08     5051.48            7   721.640000            1.877
3    Cordwainer    783.0      391.0      269.0   0.05     0.52     1.13     1444.70            2   722.350000            0.537
4       Esquire  67440.0    33746.0    36323.0  32.89    30.46    26.84   137599.19           66  2084.836212           51.137
5     Gentleman   3601.0     1799.0     1256.0   2.06     2.53     2.48     6663.07            6  1110.511667            2.476
6     Goldsmith    871.0      435.0      252.0   0.32     0.66     0.00     1558.98            1  1558.980000  

In [25]:
occs_debt_wo_dups[['Occupation', 'Total', '# of People', 'Average', '% of Total Debt']].to_excel("./FinishedSpreadsheets/occupation_debt_" + state + ".xlsx")